In [ ]:
import importlib
import sys
from urllib.request import urlretrieve

# Install drake (and underactuated).
if 'google.colab' in sys.modules and importlib.util.find_spec('underactuated') is None:
    urlretrieve(f"http://underactuated.csail.mit.edu/scripts/setup/setup_underactuated_colab.py",
                "setup_underactuated_colab.py")
    from setup_underactuated_colab import setup_underactuated
    setup_underactuated(underactuated_sha='15cfd96b0bdfd1b0c67597c24f91907776c02a6d', drake_version='0.27.0', drake_build='release')

server_args = []
if 'google.colab' in sys.modules:
  server_args = ['--ngrok_http_tunnel']
# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc_planar, zmq_url_planar, web_url_planar = start_zmq_server_as_subprocess(server_args=server_args)
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)

import numpy as np
from ipywidgets import FloatSlider, ToggleButton
from IPython.display import display, SVG, HTML
import pydot

import pydrake.all
from pydrake.all import (
    MultibodyPlant, DiagramBuilder, LinearQuadraticRegulator, Saturation, SceneGraph, Simulator, 
    WrapToSystem, AddMultibodyPlantSceneGraph, Parser, MathematicalProgram, eq, PiecewisePolynomial,
    SnoptSolver, TrajectorySource, MultibodyPositionToGeometryPose, PlanarSceneGraphVisualizer
)
from pydrake.examples.acrobot import AcrobotPlant, AcrobotGeometry
from pydrake.systems.jupyter_widgets import WidgetSystem
from pydrake.common.containers import namedview
from underactuated import FindResource
from underactuated.jupyter import running_as_notebook


Cloning into '/opt/underactuated'...

HEAD is now at 15cfd96 and again

ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.7.1 which is incompatible.
ERROR: bokeh 2.3.2 has requirement pillow>=7.1.0, but you'll have pillow 7.0.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.






Notebook Goals:

*   Function to evaluate integrated cost (ie. total cost it took to get there), can break up by state (away from target state) and actuation state -> quad cost function
*   Function to evaluate and graph time to steady state (ie. settling time)
*   Function to evaluate and graph overshoot (both types, in x and in theta)
*   Function to evaluate and graph average tilt angle



Inputs to each function:

$q[t]$ - the state ($x, \theta$) for each $t$

$u[t]$ - the actuation input ($u_x, u_\theta$) for each $t$

goal - the goal state ($x_{tf}, \theta_{tf}$)

$T$ - the total time

In [ ]:
def integrated_cost(q, u, goal, T):
  total_distance_from_target = 0
  total_actuation = 0
  goal_x = goal[0]

  for t in range(len(q)):
    cur_x = q[t][0]
    distance = (cur_x**2 + goal_x**2)**0.5
    total_distance_from_target += distance

    total_actuation += u[t][0]  # assuming u_x = u_theta since that's what Jorge said

  return total_distance_from_target/len(q), total_actuation/len(q)


In [ ]:
def settling_time(q, u, goal, T):
  """ Return None if never settled """
  settling_time = None
  goal_x = goal[0]

  for t in range(len(q) - 1, -1, -1):
    cur_x = q[t][0]
    if cur_x != goal_x:
      settling_time = t + 1
      break

  if q[-1][0] != goal[0]:
    return None

  time_steps = len(q)
  increment = T/time_steps
  settling_time *= increment
  return settling_time

In [ ]:
def overshoot(q, u, goal, T, display=False):
  """ Return None if never reached goal """
  settling_time = None
  goal_x = goal[0]
  start_x = q[0][0]
  timestep_reached_goal = None

  if start_x < goal_x:
    for t in range(len(q)):
      cur_x = q[t][0]
      if cur_x >= goal_x:
        timestep_reached_goal = t  # overestimate if not exactly equal
        break
  else:
    for t in range(len(q)):
      cur_x = q[t][0]
      if cur_x <= goal_x:
        timestep_reached_goal = t  # overestimate if not exactly equal
        break

  total_overshoot = 0
  for t in range(timestep_reached_goal, len(q), -1):
    cur_x = q[t][0]
    total_overshoot += abs(goal_x - cur_x)

  return total_overshoot/